# DREAM-tools

Install or upgrade dream-tools:

In [1]:
!pip install dream-tools --upgrade

Our convention is to import the *dreamtools* package under the alias *dt*.


In [2]:
import os
os.chdir("..")
sys.path.insert(0, os.getcwd())
import dreamtools as dt
print(dt)
print(os.getcwd())


<module 'dreamtools' from 'c:\\Users\\B031441\\Dropbox\\Programming\\dream-tools\\dreamtools\\__init__.py'>
c:\Users\B031441\Dropbox\Programming\dream-tools


In [3]:
import dreamtools as dt

## Gams Pandas
DREAM-tools provides a wrapper class *GamsPandasDatabase* that enables convertion between Pandas data structures and a Gams Database.

We can either open an existing GDX file as a GamsPandasDatabase or create new database from scratch.

### Creating and populating a new GamsPandasDatabase

To create an empty database we use the GamsPandasDatabase class.

The GamsPandasDatabase has 3 main methods for creating new Gams symbols: *create_parameter*, *create_variable*, and *create_set*. Each of these methods adds a Gams symbol to the underlying Gams Database and returns a Pandas object (or a scalar) representing the symbol.

To improve readability, we can assign some shorter names (In SnakeCase to indicate that these methods essentially work as classes):

In [4]:
db = dt.GamsPandasDatabase()
Par, Var, Set = db.create_parameter, db.create_variable, db.create_set

### Define sets

In [5]:
t = Set("t", range(2010, 2020), "Årstal")
s = Set("s", ["tjenester", "fremstilling"], "Brancher")

st = Set("st", [s, t], "Branche x år dummy")
sub = Set("sub", ["tjenester"], "Subset af brancher", domains=["s"])

one2one = Set("one2one", [(2010, 2015), (2011, 2016)], "1 til 1 mapping", domains=["t", "t"])
one2many = Set("one2many",
               [("tot", "tjenester"), ("tot", "fremstilling")],
               "1 til mange mapping", domains=["*", "s"],
               )


### Define new parameters and variables



In [6]:
gq = Par("gq", None, "Produktivitets-vækst", 0.01)
fq = Par("fp", t, "Vækstkorrektionsfaktor", (1 + 0.01) ** (t - 2010))
d = Par("d", st, "Dummy")
y = Var("y", [s, t], "Produktion")
p = Var("p", [s, t], "Pris")

### Python representation of GAMS symbols

A GAMS set is represented by a Pandas Index or a Pandas MultIndex if it has more than one dimension.

Scalars, such as *gq*, are simply Floats or Ints, while all other variable or parameters are accessed as Pandas Series:

In [7]:
print(type(s))
print(type(one2many))

print(type(gq))
print(type(p))

<class 'pandas.core.indexes.base.Index'>
<class 'pandas.core.indexes.multi.MultiIndex'>
<class 'float'>
<class 'pandas.core.series.Series'>


As they are regular Pandas Series, all the usual methods can be used (and googled). E.g. reset_index can be used to get a DataFrame from a multiindex Series:

In [8]:
p.reset_index().head()

,s,t,p
0,tjenester,2010,NaN
1,tjenester,2011,NaN
2,tjenester,2012,NaN
3,tjenester,2013,NaN
4,tjenester,2014,NaN


### Assigning to existing symbols
To change the values of the underlying Gams symbols, start by assigning values to their Pandas Series representations:

In [9]:
y["tjenester"], y["fremstilling"] = 7, 3
p[:] = 100

DREAM-tools defaults to using Pandas Series with multi-indexes rather than DataFrames.

That is because using multi-indexes greatly simplifies operations when our variables have multiple dimensions with consistent names. For example, Pandas automatically aligns the *y* and *fq* variables on their shared *t* axis without us having to do any complicated merging operations:

In [10]:
y *= fq
p * y

s             t   
tjenester     2010    700.000000
              2011    707.000000
              2012    714.070000
              2013    721.210700
              2014    728.422807
              2015    735.707035
              2016    743.064105
              2017    750.494746
              2018    757.999694
              2019    765.579691
fremstilling  2010    300.000000
              2011    303.000000
              2012    306.030000
              2013    309.090300
              2014    312.181203
              2015    315.303015
              2016    318.456045
              2017    321.640606
              2018    324.857012
              2019    328.105582
dtype: float64

Checking if a symbol name already exists in the database is as simple as

In [11]:
print("y" in db)
print("free_lunch" in db)

True
False


To figure out which type of symbol it is, we can either check which dictionary it belongs to or access the underlying Gams Symbol through the *.symbols* dictionary.

In [12]:
print("y" in db.sets)
print("y" in db.variables)
print("y" in db.parameters)

print(type(db.symbols["y"]))

False
True
False
<class 'gams.database.GamsVariable'>


Usually we only want to access the Pandas representation of a symbol. We can access those using either **\[  \]** or **.** notation.

In [13]:
print(type(db["y"]))
print(db.y is db["y"])

<class 'pandas.core.series.Series'>
True


Use **\[ \]** notation to assign a new value to an existing scalar

In [14]:
db["gq"] = 0.02

### Exporting the database to a gdx file

In [15]:
db.export("test_io.gdx")

## Working with an existing gdx file
If we already have a gdx file, such as the one we just exported, we can read that instead of creating a GamsPandasDatabase from scratch using the Gdx class:

In [16]:
gdx = dt.Gdx("test_io.gdx")

## Time series analysis

If the *Plotly* package is installed, DREAM-tools sets the plotting backend of Pandas to Plotly. That means Plotly is used by default when plotting or printing a Pandas objects instead of Matplotlib. Neither plotting packages handles multi-indexes well. To plot our *y* variable from befroe we could write:

In [17]:
y.unstack("s").plot()

We could also use the DREAM-tools plot function:

In [18]:
from dreamtools import plot
plot(y)

The plot is simply shorthand for calling DREAM-tools DataFrame followed by a plot method:

In [19]:
dt.DataFrame(y).plot()

Calling dt.DataFrame directly is especially useful if we want to inspect, manipulate or save the underlying plot data.

For more advanced examples, we open a gdx file as a GamsPandasDatabase and set it as the "reference database".
The reference database should be our baseline scenario for any analysis.

In [20]:
dt.REFERENCE_DATABASE = dt.Gdx("Q:/MAKRO/Data/PreviousSolutions/previous_dynamic_calibration")

We read another gdx file and assign it to a short name for easy access.

In [21]:
s = dt.Gdx("Q:/MAKRO/Data/PreviousSolutions/previous_dynamic_calibration2")

Let's try plotting a few national income variables:

In [22]:
plot([s.qX, s.qC, s.qG, s.qI])

Note that only the aggregate elements are plotted despite each variable being multidimensional.

That is because *dreamtools* is setup with a dictionary defining which elements should be shown for a number of sets if we haven't specified anything.

In [23]:
dt.DEFAULT_SET_AGGREGATIONS

{'a_': ['tot'],
 'a': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
 'c_': ['cTot'],
 'c': ['cBil', 'cEne', 'cVar', 'cBol', 'cTje', 'cTur'],
 'x_': ['xTot'],
 'x': ['xEne', 'xVar', 'xSoe', 'xTje', 'xTur'],
 'g_': ['gTot'],
 'g': ['g'],
 's_': ['tot'],
 'sp': ['tje', 'fre', 'byg', 'soe', 'bol', 'lan', 'ene', 'udv'],
 's': ['tje', 'fre', 'byg', 'soe', 'bol', 'lan', 'ene', 'udv', 'off'],
 'r_': ['tot'],
 'r': ['tje', 'fre', 'byg', 'soe', 'bol', 'lan', 'ene', 'udv', 'off'],
 'd_': ['tot'],
 'i_': ['iTot'],
 'i': ['IM', 'IB', 'IL'],
 'k_': ['IM', 'IB'],
 'k': ['IM', 'IB'],
 'portf_': ['NetFin'],
 'portf': ['NetFin']}

If we want other components we can simply be more specific (or supply a different *default_set_aggregations* dictionary):

In [24]:
plot(s.vC[s.c])

For a better legend, we supply a names iterable with the same length as the number of series to be plottet:

In [25]:
plot([s.qX, s.qC, s.qG, s.qI], names=["Eksport (qX)", "Privat forbrug (qC)", "Offentligt forbrug (qG)", "Investeringer (qI)"]).show()
plot(s.vC[s.c], names=list(s.c.texts))

To compare variables with our baseline (the reference database) instead of writing

```
100 * (s.vX / baseline.vX - 1), 100 * (s.vX / baseline.vX - 1) ...
```

we can use the operator "pq":

In [26]:
plot([s.vX, s.vC, s.vG, s.vI], "pq", start_year=2015, end_year=2025)

The *prt* function works identically, except instead of returning a plotly figure for plotting, it returns a string for printing.

We can set a global time period of interest using the *time* function.

4 operators are currently implemented:
- "s": show baseline side-by-side (shock, baseline)
- "q": relative multiplier (shock / baseline - 1)
- "pq": pct. relative multiplier (shock / baseline - 1) * 100
- "m": absolute multiplyer (shock - baseline)



In [27]:
from dreamtools import prt
dt.time(2017, 2025)
prt(s.vX, "s")

,vX[xTot],baseline.vX[xTot]
t,,
2017,976.289020,976.289020
2018,962.310680,962.207130
2019,952.206049,952.031343
2020,968.451771,968.300385
2021,987.076999,987.221238
2022,1004.459249,1004.944533
2023,1016.586564,1017.388690
2024,1025.172015,1026.196137
2025,1032.232029,1033.415403


Here is an example with a few more useful arguments:

In [28]:
plot([s.qY, s.qK],
        operator="m",
        start_year=2016,
        end_year=2080,
        default_set_aggregations={"s_": ["tje"], "s": ["tje"]},
        layout=dict(title = "Example graph", yaxis_title = "Difference from baseline"), # See plotly documentation for other layout options
)

### Operators (multipliers) with expressions
To get multipliers on arbitrary expressions, we can call dt.Dataframe with a database as input together with an operator and a function to be applied to the database:

In [29]:
dt.DataFrame(s, "pq", lambda s: s.pY[s.s] * s.qY[s.s], names=list(s.s.texts)).plot()

### Comparing multiple shocks and baselines

This above method of calling dt.DataFrame (or dt.plot) with a database and a function is especially useful for comparing multiple shocks. First, we import the shock and baseline databases:

In [30]:
# Import shock data
s1 = dt.Gdx("Q:/MAKRO/Data/PreviousSolutions/previous_dynamic_calibration")
s2 = dt.Gdx("Q:/MAKRO/Data/PreviousSolutions/previous_dynamic_calibration2")

# If they share a baseline, you can set it globally with
dt.REFERENCE_DATABASE = dt.Gdx("Q:/MAKRO/Data/PreviousSolutions/previous_dynamic_calibration")

# We can also set different baselines associated with each shock using
s1.reference_database = dt.Gdx("Q:/MAKRO/Data/PreviousSolutions/previous_dynamic_calibration")
s2.reference_database = dt.Gdx("Q:/MAKRO/Data/PreviousSolutions/previous_dynamic_calibration")

We can create multipliers by calling dt.DataFrame with a list of databases, an operator, and a function to be applied to each database:

In [31]:
dt.DataFrame([s1, s2], "pq", lambda s: s.qBNP, names=["Shock 1", "Shock 2"]).plot()

For even more complicated expressions involving baseline values, we can supply a function that takes two databases as input:

In [32]:
dt.plot([s1, s2], function=lambda s, b: s.qY * b.pY, names=["Shock 1", "Shock 2"])

### Age profiles
2 functions have been added specifically to plot age profiles more easily:

In [33]:
dt.age_figure_2d(s.vC_a, years=[2017, 2090], start_age=18, end_age=100)

In [34]:
dt.age_figure_3d(s.vC_a)


### Exporting plots
There are many ways to export the plots. To save a single plot as a static image, the standard plotly method works:

In [35]:
plot(s.vG).write_image(r"Output\vG.svg")

An easy way to store and share multiple plots is to export an html file

In [36]:
figures = [
    plot(s.vC),
    plot(s.vG)
]
dt.figures_to_html(figures, r"Output\example.html")

To save many figures as individual static images, we recommend collecting figures in a list or dictionary before exporting:

In [37]:
output_directory = "Output"
file_extension = ".png"
figures = dict(
    vC = plot(s.vC),
    vG = plot(s.vG),
)
for fname, fig in figures.items():
    file_path = f"{output_directory}/{fname}{file_extension}"
    dt.write_image(fig, file_path)

For png files, it is recommended to use dt.write_image instead of fig.write_image, as the former saves the image with better metadata with a size in inches (suitable for use in Word, Powerpoint, LyX etc.)